# Web Scraping Formula E

In [ ]:
# Import libraries
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import pandas as pd


In [ ]:
#Write a pathway for macbook user to run Chrome
driver = webdriver.Chrome("your_path_to_chromedriver/chromedriver")

# Open the YouTube website and search for "Maserati Folgore" videos
driver.get("https://www.youtube.com/")
time.sleep(10)



In [ ]:
# Locate the cookie acceptance button using its XPATH
cookie_button = driver.find_element(By.XPATH,'//*[@id="content"]/div[2]/div[6]/div[1]/ytd-button-renderer[2]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]')

# Click the cookie acceptance button to proceed
cookie_button.click()

time.sleep(10)

In [ ]:
# Locate the search bar element on the YouTube homepage using its XPATH
search_bar = driver.find_element(By.XPATH, '/html/body/ytd-app/div[1]/div/ytd-masthead/div[4]/div[2]/ytd-searchbox/form/div[1]/div[1]/input')

In [ ]:
# Type the query "Maserati FormulaE" into the search bar
search_bar.send_keys("Maserati FormulaE")

# Submit the search form to execute the search on YouTube
search_bar.submit()

In [26]:
# Wait for the search results to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, "contents")))


<selenium.webdriver.remote.webelement.WebElement (session="e810db38b53da3c6693bbed2c908c787", element="CCE4BE89F6E5A8CD25480AD70595273F_element_62")>

In [ ]:
# Find all the video links on the search results page that are not "People also watched"
video_links = driver.find_elements(By.CSS_SELECTOR, "#contents ytd-video-renderer #video-title:not([aria-label*='People also watched'])")


# Create an empty list to store the dataframes for each video
dfs = []

# Loop through each video link
for link in video_links:
    # Click on the link to navigate to the video page
    link.click()

    time.sleep(10)
    
    # Wait until comments are visible
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="contents"]//ytd-comment-thread-renderer')))
    
    # Extract the video title
    titleElem = driver.find_element_by_xpath('//*[@id="title"]/h1/yt-formatted-string')
    title=titleElem.text 

    # Scroll down to load all the comments on the page
    lastHeight = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)
        newHeight = driver.execute_script("return document.documentElement.scrollHeight")
        if newHeight == lastHeight:
            break
        lastHeight = newHeight

    time.sleep(5)
    # Expand any "View replies" buttons to load and scrape replies to comments
    replyButtons = driver.find_elements(By.XPATH, '//*[@id="more-replies"]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]')
    for replyButton in replyButtons:
        replyButton.location_once_scrolled_into_view
        driver.execute_script("arguments[0].click()", replyButton)
        time.sleep(2)

    # Find the updated HTML elements for comments, dates, usernames, likes, and dislikes after expanding the replies
    commentElems = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
    dateElems = driver.find_elements(By.XPATH, '//*[@id="header-author"]/yt-formatted-string/a')
    usernameElems = driver.find_elements(By.XPATH, '//*[@id="content-text"]/ancestor::ytd-comment-renderer//a[@id="author-text"]')
    likeElems = driver.find_elements(By.XPATH, '//*[@id="vote-count-middle"]')

    # Initialise lists to store scraped data
    comments = []
    dates = []
    usernames = []
    likes = []
    emojis=[]
    emojiAlt=""

    # Iterate through each comment and extract data    
    for i in range(len(commentElems)):
        emojiAlt=""
        commentText = commentElems[i].text
        date = dateElems[i].text
        username = usernameElems[i].text
        likeCount = likeElems[i].get_attribute('aria-label')

        innerImg = commentElems[i].find_elements_by_tag_name('img')
        for j in innerImg:
            if j:
                emojiAlt += j.get_attribute('alt')
        
        # Append extracted data to lists
        comments.append(commentText + " " + emojiAlt)
        dates.append(date)
        usernames.append(username)
        likes.append(likeCount)
        # dislikes.append(dislike_count)

    # Create a DataFrame from the comments and add the video title as a column
    df = pd.DataFrame({'Username':usernames,'Comment': comments, 'Date': dates,'LikeCount':likes})
    df.insert(0, "Title", title)
    
    # Append the DataFrame to the list
    dfs.append(df)
    
    # Navigate back to the search results page
    driver.back()

    # Wait for the search results to load
    wait.until(EC.presence_of_element_located((By.ID, "contents")))

# Close the web browser
driver.quit()


In [ ]:
# Concatenate all dataframes into a single dataframe
result = pd.concat(dfs)

# Export the dataframe to a CSV file
result.to_excel("FormulaE.xlsx", index=False)
